In [61]:
import pandas as pd

df = pd.read_csv('/Users/ssaqa/Desktop/flutter_projects/jaundice_detector_ml/ml_model/jaundice_dataset/chd_jaundice_published_2.csv')





In [62]:
df.head()

,patient_id,image_idx,gender,gestational_age,age(day),weight,blood(mg/dL),Treatment
0,3,0003-1.jpg,F,40,5.2,3280,3.9,0
1,3,0003-2.jpg,F,40,5.2,3280,3.9,0
2,3,0003-3.jpg,F,40,5.2,3280,3.9,0
3,35,0035-1.jpg,M,39,8.7,3760,12.2,0
4,35,0035-2.jpg,M,39,8.7,3760,12.2,0


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2235 entries, 0 to 2234
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   patient_id       2235 non-null   int64  
 1   image_idx        2235 non-null   object 
 2   gender           2235 non-null   object 
 3   gestational_age  2235 non-null   int64  
 4   age(day)         2229 non-null   float64
 5   weight           2235 non-null   int64  
 6   blood(mg/dL)     2235 non-null   float64
 7   Treatment        2235 non-null   int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 139.8+ KB


In [64]:
df.describe()

,patient_id,gestational_age,age(day),weight,blood(mg/dL),Treatment
count,2235.000000,2235.000000,2229.000000,2235.000000,2235.000000,2235.000000
mean,747.554362,38.293960,6.599192,3158.778523,11.206846,0.417450
std,420.653073,1.807409,5.703538,529.903610,5.213196,0.493249
min,1.000000,30.000000,0.100000,1690.000000,0.000000,0.000000
25%,356.000000,37.000000,2.500000,2800.000000,7.400000,0.000000
50%,794.000000,39.000000,5.000000,3180.000000,11.800000,0.000000
75%,1139.000000,40.000000,9.000000,3500.000000,15.100000,1.000000
max,1351.000000,42.000000,34.000000,5290.000000,25.700000,1.000000


In [65]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2230    False
2231    False
2232    False
2233    False
2234    False
Length: 2235, dtype: bool

In [66]:
df.isnull().sum()


patient_id         0
image_idx          0
gender             0
gestational_age    0
age(day)           6
weight             0
blood(mg/dL)       0
Treatment          0
dtype: int64

In [67]:
df.isnull().any(axis=0)

patient_id         False
image_idx          False
gender             False
gestational_age    False
age(day)            True
weight             False
blood(mg/dL)       False
Treatment          False
dtype: bool

In [68]:
df.columns

Index(['patient_id', 'image_idx', 'gender', 'gestational_age', 'age(day)',
       'weight', 'blood(mg/dL)', 'Treatment'],
      dtype='object')

In [70]:
df = df.drop(['gender', 'age(day)', 'weight'], axis=1, errors='ignore')
df.to_csv('/Users/ssaqa/Desktop/flutter_projects/jaundice_detector_ml/ml_model/jaundice_dataset/chd_jaundice_published_2.csv', index=False)


In [71]:
df["Treatment"] = df["Treatment"].map({0: "not_jaundiced", 1: "jaundiced"})
df.to_csv('/Users/ssaqa/Desktop/flutter_projects/jaundice_detector_ml/ml_model/jaundice_dataset/chd_jaundice_published_2.csv', index=False)

In [72]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

localDatasetPath = "/Users/ssaqa/Downloads/NeoJaundice/images"

image_names = df['image_idx'].tolist()

processed_images = []


def preprocess_image_tf(image_id):
    filename = f"{image_id}" 
    full_path = os.path.join(localDatasetPath, filename)

    img = tf.io.read_file(full_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = tf.cast(img, tf.float32) / 255.0
    return img

for image_id in image_names:
    try:
        processed_img = preprocess_image_tf(image_id)
        processed_images.append(processed_img.numpy())
    except Exception as e:
        print(f"❌ Error processing {image_id}: {e}")


processed_images = np.array(processed_images)
print("✅ Final shape:", processed_images.shape)

✅ Final shape: (2235, 224, 224, 3)


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np


X = processed_images  
y = df["Treatment"].map({"not_jaundiced": 0, "jaundiced": 1}).values  


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42)

print(" Shapes:")
print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)


 Shapes:
Train: (1564, 224, 224, 3) (1564,)
Validation: (335, 224, 224, 3) (335,)
Test: (336, 224, 224, 3) (336,)
